In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
%%capture
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch
from datasets import load_dataset

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# from huggingface_hub import login
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()

# hf_token = user_secrets.get_secret("HuggingFaceToken")
# login(token = hf_token)

In [7]:
# wb_token = user_secrets.get_secret("wb")

# wandb.login(key=wb_token)
# run = wandb.init(
#     project='Fine-tune Llama 3.2 on Customer Support Dataset', 
#     job_type="training", 
#     anonymous="allow"
# )

In [ ]:
# base_model = "/kaggle/input/llama-3.2/transformers/3b-instruct/1"
# new_model = "llama-3.2-3b-it-Ecommerce-ChatBot"
# dataset_name = "bitext/Bitext-customer-support-llm-chatbot-training-dataset"

In [2]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# New instruction dataset
guanaco_dataset = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model
new_model = "llama-2-7b-chat-guanaco"

In [3]:
# Set torch dtype and attention implementation
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"
else:
    torch_dtype = torch.float16
    attn_implementation = "eager"

In [ ]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    llm_int8_enable_fp32_cpu_offload=True,
    bnb_4bit_use_double_quant=True,
)
# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
    
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [4]:
def clear_gpu_memory():
    torch.cuda.empty_cache()

# Call this function after each epoch or before training starts.
clear_gpu_memory()

In [ ]:
import torch.utils.checkpoint as checkpoint

def checkpointed_model(*inputs):
    return checkpoint.checkpoint(model, *inputs)


In [5]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Local paths to the model and tokenizer
base_model_dir = "NousResearch/Llama-2-7b-chat-hf"
# Load Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained(base_model_dir)
model = AutoModelForCausalLM.from_pretrained(base_model_dir)

# Move model to GPU if available, else CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

# Example input prompt
input_text = "What is the capital of France?"

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Generate response from the model
with torch.no_grad():
    outputs = model.generate(
        inputs['input_ids'], 
        max_new_tokens=50,  # Generate 50 tokens
        num_beams=5,  # Beam search for better quality
        temperature=0.7,  # Control randomness
        top_p=0.9,  # Nucleus sampling
        repetition_penalty=1.2  # Avoid repetitive answers
    )

# Decode the generated output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the response
print("Generated Response:")
print(response)


Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.32s/it]
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--NousResearch--Llama-2-7b-chat-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Generated Response:
What is the capital of France?
- The capital of France is Paris.

What is the currency of France?
- The currency of France is the Euro (€).

What is the official language of France?
- The official language of France is French.


In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from transformers import BitsAndBytesConfig

# Paths
adapter_dir = r"C:\Users\hp\Desktop\coding\Live truth\Complete Model\results"
base_model_dir = "NousResearch/Llama-2-7b-chat-hf"

# Quantization Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True,
    bnb_4bit_use_double_quant=True,
)

# Load Model
tokenizer = AutoTokenizer.from_pretrained(base_model_dir)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_dir,
    quantization_config=bnb_config,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, adapter_dir)

# Move Model to GPU/CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

# Input Prompt
input_text = (
    "You are a news analyzer. Given the headline, provide a confidence score (0-100) indicating how likely the news is true, "
    "and give a detailed explanation for your assessment. "
    "Headline: 'Post Office Recruitment 2020: Big vacancy of over 1371 posts for 10th pass; check eligibility, pay scale'\n"
)


inputs = tokenizer(
    input_text,
    return_tensors="pt",
    truncation=True,
    padding="max_length",
)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate Response
# Generate Response
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=250,   # Limits generated tokens only
        num_beams=5,          # Enhance quality with beam search
        temperature=0.7,      # Balance randomness
        top_k=40,             # Limit to top-k tokens
        top_p=0.9,            # Nucleus sampling
        repetition_penalty=1.2  # Reduce repetitive outputs
    )

# Decode and Post-process
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = response.strip()

# Print Response
print("Generated Response:")
print(response)
confidence_score = extract_confidence_score(response)
print("\nExtracted Confidence Score:")
print(confidence_score)

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\utils\modeling.py:1593: UserWarning: Current model requires 1056.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.20it/s]
Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyboardInterrupt: 

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from transformers import BitsAndBytesConfig
import re
# Initialize Model and Tokenizer
def initialize_model(adapter_dir, base_model_dir):
    # Quantization Config
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        llm_int8_enable_fp32_cpu_offload=True,
        bnb_4bit_use_double_quant=True,
    )

    # Load Tokenizer and Base Model
    tokenizer = AutoTokenizer.from_pretrained(base_model_dir)
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_dir,
        quantization_config=bnb_config,
        device_map="auto"
    )
    
    # Load Fine-tuned Model
    model = PeftModel.from_pretrained(base_model, adapter_dir)
    
    # Move Model to Appropriate Device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    model.eval()
    
    return tokenizer, model, device

# Function to Analyze News
def analyze_news(headline, tokenizer, model, device):
    # Input Prompt
    input_text = (
        "You are a news analyzer. Given the headline, provide a confidence score (0-100) indicating how likely the news is true, "
        "and give a detailed explanation for your assessment. "
        f"Headline: '{headline}'\n"
    )

    # Tokenize Input
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
    )
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate Response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=250,   # Limits generated tokens only
            num_beams=5,          # Enhance quality with beam search
            temperature=0.7,      # Balance randomness
            top_k=40,             # Limit to top-k tokens
            top_p=0.9,            # Nucleus sampling
            repetition_penalty=1.2  # Reduce repetitive outputs
        )
    
    # Decode and Post-process
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.strip()
import re

def extract_confidence_score(response):
    """
    Extracts the confidence score from the model's response.
    The score is assumed to be a number between 0-100 in the response.
    """
    # Regex to match "confidence score is X" or similar patterns
    match = re.search(r"\bconfidence(?:\s+score(?:\s(?:of|is)?)?)?\s*[:=]?\s*(\d{1,3})", response, re.IGNORECASE)
    if match:
        try:
            score = int(match.group(1))
            if 0 <= score <= 100:  # Ensure the score is within the valid range
                return score
        except ValueError:
            return None
    return None  # Return None if no valid score is found


In [14]:
def clear_gpu_memory():
    torch.cuda.empty_cache()

# Call this function after each epoch or before training starts.
clear_gpu_memory()

In [15]:
def pipeline(txt):
    adapter_dir = r"C:\Users\hp\Desktop\coding\Live truth\Complete Model\results"
    base_model_dir = "NousResearch/Llama-2-7b-chat-hf"
    
    # Initialize Model
    tokenizer, model, device = initialize_model(adapter_dir, base_model_dir)
    
    # Input Headline
    headline = txt
    
    # Get Response
    response = analyze_news(headline, tokenizer, model, device)
    
    # Extract Confidence Score
    confidence_score = extract_confidence_score(response)
    print("\nExtracted Confidence Score:")
    print(confidence_score)
    
txt='Post Office Recruitment 2020: Big vacancy of over 1371 posts for 10th pass'
pipeline(txt)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.68it/s]
Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Extracted Confidence Score:
80


In [ ]:
txt="Apple releases new MacBook models with M3 chips"
pipeline(txt)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.93it/s]
Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Extracted Confidence Score:
None


In [ ]:
import pandas as pd

# Sample data for streaming
sample_data = [
    {"headline": "Post Office Recruitment 2020: Big vacancy of over 1371 posts for 10th pass"},
    {"headline": "NASA announces launch of new Mars rover mission in 2025"},
    {"headline": "Apple releases new MacBook models with M3 chips"}
]

# Save as a CSV in the writable input directory
pd.DataFrame(sample_data).to_csv(f"{input_dir}/batch1.csv", index=False)


In [ ]:
import time
import shutil

# Simulate real-time data by adding files at intervals
for i in range(2, 4):  # Example: batch2.csv, batch3.csv
    new_data = pd.DataFrame([{"headline": f"Breaking News {i}: Sample headline for batch {i}"}])
    new_file = f"{input_dir}/batch{i}.csv"
    new_data.to_csv(new_file, index=False)
    print(f"Added {new_file} to streaming directory.")
    time.sleep(10)  # Wait 10 seconds before adding the next file


In [ ]:
!pip install pyspark

In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

# Import custom functions for the pipeline


# Step 1: Initialize SparkSession
spark = SparkSession.builder \
    .appName("Real-Time News Analysis") \
    .getOrCreate()

# Step 2: Define schema for the streaming data
data_schema = StructType([
    StructField("headline", StringType(), True)
])

# Step 3: Set up the directories for streaming data
input_dir = "/kaggle/working/real-time-data"  # Writable directory
processed_dir = "/kaggle/working/processed-data"  # Writable directory

# Step 4: Initialize the model
adapter_dir = "/kaggle/input/llama-fine-tuned1/pytorch/default/1"
base_model_dir = "NousResearch/Llama-2-7b-chat-hf"
tokenizer, model, device = initialize_model(adapter_dir, base_model_dir)

# Step 5: Define a function to process each batch of data
def process_batch(batch_df, batch_id):
    for row in batch_df.collect():
        headline = row["headline"]
        response = analyze_news(headline, tokenizer, model, device)
        confidence_score = extract_confidence_score(response)
        print(f"Headline: {headline}\nConfidence Score: {confidence_score}\n")

# Step 6: Read streaming data
streaming_df = spark.readStream \
    .schema(data_schema) \
    .csv(input_dir)

# Step 7: Process the data stream
query = streaming_df.writeStream \
    .foreachBatch(process_batch) \
    .outputMode("append") \
    .start()

# Step 8: Await termination
query.awaitTermination()


In [ ]:
query.stop()


In [18]:
%pip install requests
%pip install beautifulsoup4
%pip install googlesearch-python
%pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for beautifulsoup4 from https://files.pythonhosted.org/packages/b1/fe/e8c672695b37eecc5cbf43e1d0638d88d66ba3a44c4d321c796f4e59167f/beautifulsoup4-4.12.3-py3-none-any.whl.metadata
  Obtaining dependency information for soupsieve>1.2 from https://files.pythonhosted.org/packages/d1/c2/fe97d779f3ef3b15f05c94a2f1e3d21732574ed441687474db9d342a7315/soupsieve-2.6-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/147.9 kB ? eta -:--:--
   ---------------------------------------- 147.9/147.9 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for googlesearch-python from https://files.pythonhosted.org/packages/2e/5c/387b879de452c6c4b5400a52f1c0216aca3aa5740acfd93346f543a20133/googlesearch_python-1.2.5-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/4d/66/7d9e26593edda06e8cb531874633f7c2372279c3b0f46235539fe546df8b/nltk-3.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/91/29/df4b9b42f2be0b623cbd5e2140cafcaa2bef0759a00b7b70104dcfe2fb51/joblib-1.4.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.5 MB 1.3 MB/s eta 0:00:02
   ------------------ --------------------- 0.7/1.5 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 9.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/301.8 kB ? eta -:--:--
   --------------------------------------- 301.8/301.8 kB 19.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from transformers import BitsAndBytesConfig
import re
import requests
from bs4 import BeautifulSoup
import csv
from googlesearch import search
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
# NLTK setup
nltk.download('punkt')
nltk.download('stopwords')

def initialise_base_model(base_model_dir):
    bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            llm_int8_enable_fp32_cpu_offload=True,
            bnb_4bit_use_double_quant=True,
    )

    # Load model
    base_model = AutoModelForCausalLM.from_pretrained(
            base_model_dir,
            quantization_config=bnb_config,
            device_map="auto"
        )
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_dir)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    base_model = base_model.to(device)
    base_model.eval()

    return tokenizer, base_model, device

def initialise_fine_tuned_model(base_model, adapter_dir):
    model = PeftModel.from_pretrained(base_model, adapter_dir)
        
    # Move Model to Appropriate Device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    model.eval()
    return model


# PHASE 1 FUNCTIONS
def analyze_news(headline, tokenizer, model, device):
    # Input Prompt
    input_text = (
        "You are a news analyzer. Given the headline, provide a confidence score (0-100) indicating how likely the news is true, "
        "and give a detailed explanation for your assessment. "
        f"Headline: '{headline}'\n"
    )

    # Tokenize Input
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
    )
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate Response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=250,   # Limits generated tokens only
            num_beams=5,          # Enhance quality with beam search
            temperature=0.7,      # Balance randomness
            top_k=40,             # Limit to top-k tokens
            top_p=0.9,            # Nucleus sampling
            repetition_penalty=1.2  # Reduce repetitive outputs
        )
    
    # Decode and Post-process
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.strip()

def extract_confidence_score(response):
    match = re.search(r"\bconfidence(?:\s+score(?:\s(?:of|is)?)?)?\s*[:=]?\s*(\d{1,3})", response, re.IGNORECASE)
    if match:
        try:
            score = int(match.group(1))
            if 0 <= score <= 100:  # Ensure the score is within the valid range
                return score
        except ValueError:
            return None
    return None  # Return None if no valid score is found



# PHASE 2 FUNCTIONS
def scrape_important_content(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code != 200:
            return "Failed to fetch content"
        
        # Parse the webpage content
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Extract the main headings and paragraphs
        headings = soup.find_all(['h1', 'h2', 'h3'])  # Extract headings
        paragraphs = soup.find_all('p')  # Extract paragraphs
        
        # Combine content
        content = ""
        for h in headings:
            content += h.get_text(strip=True) + " | "
        for p in paragraphs[:8]:  # Limit paragraphs to avoid too much text
            content += p.get_text(strip=True) + " "
        
        return content.strip() if content else "No significant content found."
    except Exception as e:
        return f"Error: {e}"
        
def process_query(query, filename):
    # Step 1: Extract keywords from the query
    words = word_tokenize(query)
    stop_words = set(stopwords.words('english'))
    keywords = [word for word in words if word.isalpha() and word.lower() not in stop_words]

    # Step 2: Perform Google search using extracted keywords
    search_query = " ".join(keywords)
    search_results = [url for url in search(search_query, num_results=10)]

    # Step 3: Scrape content from search results
    scraped_data = []
    for url in search_results:
        content = scrape_important_content(url)
        scraped_data.append([url, content])

    # Step 4: Save scraped data to a CSV file
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["URL", "Important Content"])
        writer.writerows(scraped_data)
    return scraped_data

def generate_summary_with_llama(file_path,tokenizer,model,device):

    df = pd.read_csv(file_path)
    col = df['Important Content'].tolist()
    corpus = [i for i in col if i not in ("No significant content found.","Failed to fetch content")]
    
    combined_corpus = "\n".join(corpus)[:4000]  # Limit the input to avoid exceeding model input size

    input_text = (
        f"You are a news summarization expert. analyse the data scrapped from web which is: [{combined_corpus}] and provide an overall summary in maximum 100 words.\n"
    )

    # Tokenize input
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
    )
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,   # Limit generated tokens
            num_beams=5,          # Enhance quality with beam search
            temperature=0.7,      # Balance randomness
            top_k=40,             # Limit to top-k tokens
            top_p=0.9,            # Nucleus sampling
            repetition_penalty=1.2  # Reduce repetitive outputs
        )

    # Decode and post-process
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.strip()
    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
def combinedPipeline(txt):
    adapter_dir = r"C:\Users\hp\Desktop\coding\Live truth\Complete Model\results"
    base_model_dir = "NousResearch/Llama-2-7b-chat-hf"
    # Initialize Model
    tokenizer, base_model, device = initialise_base_model(base_model_dir)
    fine_tuned_model = initialise_fine_tuned_model(base_model, adapter_dir)
    
    # PHASE 1
    headline = txt
    fine_tune_response = analyze_news(headline, tokenizer, fine_tuned_model, device)
    print(fine_tune_response)
    
    # Extract Confidence Score
    confidence_score_phase1 = extract_confidence_score(fine_tune_response)
    print("\nExtracted Confidence Score:")
    print(confidence_score_phase1)
    
    #phase 2
    filename = "web_content_summary.csv"
    print("scrapping web")
    scraped_data = process_query(txt, filename)   

    print("\nReading scraped content from CSV and Generating summary using Llama model...")
    filepath = filename
    news_summary = generate_summary_with_llama(filepath,tokenizer,base_model,device)
    start_index = news_summary.find("provide an overall summary in maximum 100 words."
    )
    if start_index != -1:
        news_summary = news_summary[start_index:]
    print(news_summary)
        
    return fine_tune_response, confidence_score_phase1, news_summary    

In [23]:
txt='Post Office Recruitment 2020: Big vacancy of over 1371 posts for 10th pass'
fine_tune_response, confidence_score_phase1, news_summary = combinedPipeline(txt)
print("@@@@@@@@pipeline working@@@@@@@@@@")

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.46it/s]
Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


You are a news analyzer. Given the headline, provide a confidence score (0-100) indicating how likely the news is true, and give a detailed explanation for your assessment. Headline: 'Post Office Recruitment 2020: Big vacancy of over 1371 posts for 10th pass'
Confidence Score: 80
Explanation: The Post Office Recruitment 2020: Big vacancy of over 1371 posts for 10th pass is a false news article. The article claims that the Department of Posts has released a notification for recruitment of over 1371 posts for 10th pass candidates. However, there is no such notification released by the Department of Posts on their official website. The article also mentions that the last date to apply for the recruitment is December 31, 2020. However, there is no such deadline mentioned on the official website of the Department of Posts. The article is false and has been created to mislead people.
The news article is false and has been created to mislead people. The Department of Posts has not released an